**[Geospatial Analysis Home Page](https://www.kaggle.com/learn/geospatial-analysis)**

---


# Introduction

You are a Starbucks big data analyst ([that’s a real job!](https://www.forbes.com/sites/bernardmarr/2018/05/28/starbucks-using-big-data-analytics-and-artificial-intelligence-to-boost-performance/#130c7d765cdc)) looking to find the next store into a [Starbucks Reserve Roastery](https://www.businessinsider.com/starbucks-reserve-roastery-compared-regular-starbucks-2018-12#also-on-the-first-floor-was-the-main-coffee-bar-five-hourglass-like-units-hold-the-freshly-roasted-coffee-beans-that-are-used-in-each-order-the-selection-rotates-seasonally-5).  These roasteries are much larger than a typical Starbucks store and have several additional features, including various food and wine options, along with upscale lounge areas.  You'll investigate the demographics of various counties in the state of California, to determine potentially suitable locations.

<center>
<img src="https://i.imgur.com/BIyE6kR.png" width="450"><br/><br/>
</center>

Before you get started, run the code cell below to set everything up.

* 這裡的case以品牌特殊實驗店展店為題

In [54]:
import math
import pandas as pd
import geopandas as gpd
#from geopandas.tools import geocode            # What you'd normally run
from learntools.geospatial.tools import geocode # Just for this exercise

import folium 
from folium import Marker
from folium.plugins import MarkerCluster

from learntools.core import binder
binder.bind(globals())
from learntools.geospatial.ex4 import *

You'll use the `embed_map()` function from the previous exercise to visualize your maps.

In [55]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Exercises

### 1) Geocode the missing locations.

Run the next code cell to create a DataFrame `starbucks` containing Starbucks locations in the state of California.

* geocode : 從地理座標位置 <---> 地址，地標, based on google map, Bing Maps, Baidu Maps
* provide name / adress (Python string)
* API provider, goole map, BIng Maps, Baidu Maps,...
    * Here we use OpenStreetMap
* return 
    * geometry ( latitude, longitude )
    * adress ( full adress )


In [56]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("../input/geospatial-learn-course-data/starbucks_locations.csv")
display(starbucks.head(), type(starbucks))

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


pandas.core.frame.DataFrame

Most of the stores have known (latitude, longitude) locations.  But, all of the locations in the city of Berkeley are missing.

In [57]:
# How many rows in each column have missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


Use the code cell below to fill in these values with the OpenStreetMap Nominatim geocoder.

Note that in the tutorial, we used `geocode()` (from `geopandas.tools`) to geocode values, and this is what you can use in your own projects outside of this micro-course.  

In this exercise, you will use a slightly different function `geocode()` (from `learntools.geospatial.tools`).  This function was imported at the top of the notebook and works identically to the function from GeoPandas.

So, in other words, as long as: 
- you don't change the import statements at the top of the notebook, and 
- you call the geocoding function as `geocode()` in the code cell below, 

your code will work as intended!

In [58]:
# Your code here
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None



rows_with_missing[['Longitude','Latitude','geometry']] = rows_with_missing.\
apply(lambda x : my_geocoder(x['Address']), axis=1)
display(rows_with_missing)

for c in ['Longitude','Latitude']:
    print(starbucks[starbucks["City"]=="Berkeley"][str(c)])
    starbucks.loc[starbucks["City"]=="Berkeley",str(c)] = rows_with_missing[str(c)].values

# Check your answer
q_1.check()

,Store Number,Store Name,Address,City,Longitude,Latitude,geometry
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,37.868839,-122.268230,POINT (-122.2682302 37.8688395)
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,37.891383,-122.280050,POINT (-122.28005 37.891383)
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,37.880907,-122.269679,POINT (-122.2696792 37.8809073)
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,37.855947,-122.259370,POINT (-122.259369635073 37.8559472)
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,37.870410,-122.266079,POINT (-122.266079 37.87041)


153   NaN
154   NaN
155   NaN
156   NaN
157   NaN
Name: Longitude, dtype: float64
153   NaN
154   NaN
155   NaN
156   NaN
157   NaN
Name: Latitude, dtype: float64


<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> Did you use the nominatim geocoder to fill in the values?

In [9]:
# Line below will give you solution code
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
def my_geocoder(row):
    point = geocode(row, provider='nominatim').geometry[0]
    return pd.Series({'Longitude': point.x, 'Latitude': point.y})

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)

```

### 2) View Berkeley locations.

Let's take a look at the locations you just found.  Visualize the (latitude, longitude) locations in Berkeley in the OpenStreetMap style. 

In [64]:
# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Your code here: Add a marker for each Berkeley location
for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)

# Uncomment to see a hint
q_2.a.hint()

# Show the map
embed_map(m_2, 'q_2.html')

# it might be a wrong answer due to wrong geo-encoding

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Use `folium.Marker()` to add a location for each entry in `starbucks[starbucks["City"]=='Berkeley']`.

In [62]:
# Get credit for your work after you have created a map
q_2.a.check()

# Uncomment to see our solution (your code may look different!)
q_2.a.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Thank you for creating a map!</span>

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
# Add a marker for each Berkeley location
for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)

```

Considering only the five locations in Berkeley, how many of the (latitude, longitude) locations seem potentially correct (are located in the correct city)?

In [ ]:
# View the solution
q_2.b.solution()

### 3) Consolidate your data.

Run the code below to load a GeoDataFrame `CA_counties` containing the name, area (in square kilometers), and a unique id (in the "GEOID" column) for each county in the state of California.  The "geometry" column contains a polygon with county boundaries.

* JOIN with GeoDataFrame
* use gpd.GeoDataFrame.merge()
* JOIN ON string is simple
* Spatial join : 
    * based on the spatial relationship between onthes in **geometry**
    * use gpd.sjoin()
    * Point vs Polygon -> ? 
    * [other shape merging... check the doc](http://geopandas.org/reference/geopandas.sjoin.html)

In [65]:
CA_counties = gpd.read_file("../input/geospatial-learn-course-data/CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.head()

,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,POLYGON ((-120.6555981820125 39.69356523820564...
1,6067,Sacramento County,2575.258262,POLYGON ((-121.1885841398944 38.71431311442083...
2,6083,Santa Barbara County,9813.817958,(POLYGON ((-120.5819095703916 34.0985617276066...
3,6009,Calaveras County,2685.626726,POLYGON ((-120.6309460064869 38.34110512435024...
4,6111,Ventura County,5719.321379,(POLYGON ((-119.6363143545649 33.2730446943631...


Next, we create three DataFrames:
- `CA_pop` contains an estimate of the population of each county.
- `CA_high_earners` contains the number of households with an income of at least $150,000 per year.
- `CA_median_age` contains the median age for each county.

In [66]:
CA_pop = pd.read_csv("../input/geospatial-learn-course-data/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("../input/geospatial-learn-course-data/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("../input/geospatial-learn-course-data/CA_county_median_age.csv", index_col="GEOID")

Use the next code cell to join the `CA_counties` GeoDataFrame with `CA_pop`, `CA_high_earners`, and `CA_median_age`.

Name the resultant GeoDataFrame `CA_stats`, and make sure it has 8 columns: "GEOID", "name", "area_sqkm", "geometry", "population", "high_earners", and "median_age".  Also, make sure the CRS is set to `{'init': 'epsg:4326'}`.

In [69]:
# Your code here
# df, df
CA_stats = cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
# gdf, df
CA_stats = CA_counties.merge(cols_to_add, on="GEOID")

display(CA_stats)
# Check your answer
q_3.check()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age
0,6091,Sierra County,2491.995494,POLYGON ((-120.6555981820125 39.69356523820564...,2987,111,55.0
1,6067,Sacramento County,2575.258262,POLYGON ((-121.1885841398944 38.71431311442083...,1540975,65768,35.9
2,6083,Santa Barbara County,9813.817958,(POLYGON ((-120.5819095703916 34.0985617276066...,446527,25231,33.7
3,6009,Calaveras County,2685.626726,POLYGON ((-120.6309460064869 38.34110512435024...,45602,2046,51.6
4,6111,Ventura County,5719.321379,(POLYGON ((-119.6363143545649 33.2730446943631...,850967,57121,37.5
5,6037,Los Angeles County,12305.376879,(POLYGON ((-118.6676142121397 33.4774937689834...,10105518,501413,36.0
6,6097,Sonoma County,4578.952090,POLYGON ((-122.9350733556339 38.31395694932695...,499942,32713,41.4
7,6031,Kings County,3604.052342,POLYGON ((-119.9589376692871 36.25547297916683...,151366,2943,31.5
8,6073,San Diego County,11721.342229,POLYGON ((-117.4374379823408 33.17953480111068...,3343364,194676,35.4
9,6061,Placer County,3890.821444,POLYGON ((-121.0654491573255 39.00653814848021...,393149,28334,41.6


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [67]:
# Lines below will give you a hint or solution code
#q_3.hint()
q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
CA_stats = CA_counties.merge(cols_to_add, on="GEOID")

```

Now that we have all of the data in one place, it's much easier to calculate statistics that use a combination of columns.  Run the next code cell to create a "density" column with the population density.

In [70]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]

### 4) Which counties look promising?

Collapsing all of the information into a single GeoDataFrame also makes it much easier to select counties that meet specific criteria.

Use the next code cell to create a GeoDataFrame `sel_counties` that contains a subset of the rows (and all of the columns) from the `CA_stats` GeoDataFrame.  In particular, you should select counties where:
- there are at least 100,000 households making \$150,000 per year,
- the median age is less than 38.5, and
- the density of inhabitants is at least 285 (per square kilometer).

Additionally, selected counties should satisfy at least one of the following criteria:
- there are at least 500,000 households making \$150,000 per year,
- the median age is less than 35.5, or
- the density of inhabitants is at least 1400 (per square kilometer).

In [82]:
# Your code here
# target_mask
from functools import reduce
rich_area = CA_stats['high_earners'] > 100000
yonung_area = CA_stats['median_age'] < 38.5
high_density_area = CA_stats['density'] > 285
golden_area = reduce(lambda x, y : x & y, [rich_area, yonung_area, high_density_area])
single_opt_young = CA_stats.median_age < 35.5 
single_opt_high_density = CA_stats.density > 1400
single_opt_rich = CA_stats.high_earners > 500000
stillOk_area = reduce(lambda x, y : x | y, [single_opt_young, single_opt_high_density, single_opt_rich])

sel_counties = CA_stats[golden_area & stillOk_area]
display(sel_counties)
# Check your answer
q_4.check()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
5,6037,Los Angeles County,12305.376879,(POLYGON ((-118.6676142121397 33.4774937689834...,10105518,501413,36.0,821.227834
8,6073,San Diego County,11721.342229,POLYGON ((-117.4374379823408 33.17953480111068...,3343364,194676,35.4,285.237299
10,6075,San Francisco County,600.588247,(POLYGON ((-122.6002492453396 37.8024879332407...,883305,114989,38.3,1470.733077


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [71]:
# Lines below will give you a hint or solution code
#q_4.hint()
# q_4.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
sel_counties = CA_stats[((CA_stats.high_earners > 100000) &
                         (CA_stats.median_age < 38.5) &
                         (CA_stats.density > 285) &
                         ((CA_stats.median_age < 35.5) |
                         (CA_stats.density > 1400) |
                         (CA_stats.high_earners > 500000)))]

```

### 5) How many stores did you identify?

When looking for the next Starbucks Reserve Roastery location, you'd like to consider all of the stores within the counties that you selected.  So, how many stores are within the selected counties?

To prepare to answer this question, run the next code cell to create a GeoDataFrame `starbucks_gdf` with all of the starbucks locations.

In [83]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}
display(starbucks_gdf, type(starbucks_gdf))

,Store Number,Store Name,Address,City,Longitude,Latitude,geometry
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51,POINT (-117.4 34.51)
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76 34.16)
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15,POINT (-118.76 34.15)
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87,POINT (-117.75 33.87)
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79,POINT (-122.28 37.79)
...,...,...,...,...,...,...,...
2816,14071-108147,Hwy 20 & Tharp - Yuba City,"1615 Colusa Hwy, Ste 100 Yuba City CA",Yuba City,-121.64,39.14,POINT (-121.64 39.14)
2817,9974-98559,"Yucaipa & Hampton, Yucaipa","31364 Yucaipa Blvd., A Yucaipa CA",Yucaipa,-117.12,34.03,POINT (-117.12 34.03)
2818,79654-108478,Vons - Yucaipa #1796,33644 YUCAIPA BLVD YUCAIPA CA,YUCAIPA,-117.07,34.04,POINT (-117.07 34.04)
2819,6438-245084,Yucaipa & 6th,"34050 Yucaipa Blvd., 200 Yucaipa CA",Yucaipa,-117.06,34.03,POINT (-117.06 34.03)


geopandas.geodataframe.GeoDataFrame

So, how many stores are in the counties you selected?

In [93]:
# Fill in your answer
cadidate_store = gpd.sjoin(sel_counties, starbucks_gdf)
num_stores = len(cadidate_store)

display(num_stores, 
        type(num_stores),
        cadidate_store.sample(5),
        'missing geometries...',
        cadidate_store.isnull().sum()
       )
# Check your answer
q_5.check()

1043

int

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density,index_right,Store Number,Store Name,Address,City,Longitude,Latitude
8,6073,San Diego County,11721.342229,POLYGON ((-117.4374379823408 33.17953480111068...,3343364,194676,35.4,285.237299,566,9192-151004,Vons-Encinitas #2859,453 Sante Fe Dr Encinitas CA,Encinitas,-117.27,33.05
10,6075,San Francisco County,600.588247,(POLYGON ((-122.6002492453396 37.8024879332407...,883305,114989,38.3,1470.733077,2084,24218-233045,Target Express - Bush Street,225 Bush St San Francisco CA,San Francisco,-122.40,37.79
5,6037,Los Angeles County,12305.376879,(POLYGON ((-118.6676142121397 33.4774937689834...,10105518,501413,36.0,821.227834,1283,28507-249685,Foothill & Madison,860 West Foothill Blvd Monrovia CA,Monrovia,-118.02,34.15
8,6073,San Diego County,11721.342229,POLYGON ((-117.4374379823408 33.17953480111068...,3343364,194676,35.4,285.237299,580,73443-23069,Albertsons-Escondido #6713,1509 E Valley Parkway Escondido CA,Escondido,-117.06,33.13
5,6037,Los Angeles County,12305.376879,(POLYGON ((-118.6676142121397 33.4774937689834...,10105518,501413,36.0,821.227834,1148,9230-154245,Grand & 1st (Grand Park),"217 N. Hill Street, Elleven Condominium Los An...",Los Angeles,-118.25,34.06


'missing geometries...'

GEOID           0
name            0
area_sqkm       0
geometry        0
population      0
high_earners    0
median_age      0
density         0
index_right     0
Store Number    0
Store Name      0
Address         0
City            0
Longitude       0
Latitude        0
dtype: int64

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [85]:
# Lines below will give you a hint or solution code
#q_5.hint()
q_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(locations_of_interest)

```

### 6) Visualize the store locations.

Create a map that shows the locations of the stores that you identified in the previous question.

In [94]:
# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)

# Your code here: show selected store locations
# for idx, row in cadidate_store.iterrows():
#     Marker([row['Latitude'], row['Longitude']]).add_to(m_6)

    
mc = MarkerCluster()


for idx, row in cadidate_store.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))

m_6.add_child(mc)
# Uncomment to see a hint
#q_6.hint()

# Show the map
embed_map(m_6, 'q_6.html')

### use Marker Cluster to avoid 1k+ points shown

In [91]:
# Get credit for your work after you have created a map
q_6.check()

# Uncomment to see our solution (your code may look different!)
q_6.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Thank you for creating a map!</span>

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
# Show selected store locations
mc = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))
        
m_6.add_child(mc)

```

# Keep going

Learn about how **[proximity analysis](https://www.kaggle.com/alexisbcook/proximity-analysis)** can help you to understand the relationships between points on a map.

---
**[Geospatial Analysis Home Page](https://www.kaggle.com/learn/geospatial-analysis)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*